Write code to classify two famous celebrities:
We need to prepare the data to do so and also prepare the model as well.

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
class MyCLRuleMonitor(tf.keras.callbacks.Callback):
  def __init__(self, CL):
    super(MyCLRuleMonitor).__init__()
    self.CL = CL

  def on_epoch_end(self, epoch, logs=None):
    trainScore = logs["accuracy"]
    testScore = logs["val_accuracy"]

    if testScore > trainScore and testScore >= self.CL:
      self.model.stop_training = True

Steps for Model creation:
0. Data Preprocessing
    i. Image Generators
    ii. Splitting Train and Test data from Generators
1. Model architecting + Prepro
    i. CNN + ANN layers creation
2. Model Compilation
3. Model Training 
4. Model Evaluation
5. Model Testing / Deploy

In [4]:
# ImageGenerators:
training_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
testing_image_generator= tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.0)

In [6]:
training_image_data = training_image_generator.flow_from_directory('celeb/data/training',
                                                                   batch_size = 20,
                                                                   class_mode = 'binary',
                                                                   target_size= (64,64))

testing_image_data = training_image_generator.flow_from_directory('celeb/data/validation',
                                                                   batch_size = 20,
                                                                   class_mode = 'binary',
                                                                   target_size= (64,64))

Found 31 images belonging to 2 classes.
Found 22 images belonging to 2 classes.


In [8]:
# Model creation:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3,3), input_shape = training_image_data.image_shape , activation = 'relu', padding = 'same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

model.add(tf.keras.layers.Conv2D(32, (3,3), input_shape = training_image_data.image_shape, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

model.add(tf.keras.layers.Flatten())

# FC:
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))

model.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

model.summary()

c:\Users\thedo\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 7200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │       921,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 981,537 (3.74 MB)

 Trainable params: 981,537 (3.74 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics= ['accuracy'])

In [10]:
model.fit(training_image_data, 
          validation_data= testing_image_data,
          epochs = 200,
          validation_steps= (len(testing_image_data.filenames) // testing_image_data.batch_size),
          steps_per_epoch = (len(training_image_data.filenames) // training_image_data.batch_size),
          callbacks = MyCLRuleMonitor(0.7))

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 0.7000 - loss: 0.6889 - val_accuracy: 0.4500 - val_loss: 0.7255
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7273 - loss: 0.6278 - val_accuracy: 0.5000 - val_loss: 0.8918
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6364 - loss: 0.6858 - val_accuracy: 0.4500 - val_loss: 0.7285
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6000 - loss: 0.6266 - val_accuracy: 0.8000 - val_loss: 0.6699


In [ ]:
# Testing | Deployment
# Read the image - convert to np array - add dimention using np - get prediction - change prediction aproximation to value

image = tf.keras.preprocessing.image.load_img('celeb/data/validation/salman/gettyimages-94244198-1024x1024.jpg', target_size = (64,64))

image_arr = tf.keras.preprocessing.image.img_to_array(image)

np_image_arr = np.expand_dims(image_arr, axis = 0)

predictions = model.predict(np_image_arr)

# 2. Find the index of the class with the highest probability
predicted_class_index = np.argmax(predictions) #

# 3. Create a reverse mapping dictionary to get the class name from the index
# Original: {'class_name': index} -> Reversed: {index: 'class_name'}
class_names_map = {v: k for k, v in training_image_data.class_indices.items()}

# 4. Use the index to look up the actual class name
predicted_class_name = class_names_map[predicted_class_index]

# 5. Print the result
print(f"The predicted celeb name is: {predicted_class_name}") #


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


array([[1.0241885e-12]], dtype=float32)